In [17]:
import argparse
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import ogb
!python -c "import ogb; print(ogb.__version__)"

1.2.6


In [18]:
import pandas as pd
import os
import numpy as np
base_path = './dataset/ogbl_ppa/raw'
node_feat_csv = pd.read_csv(os.path.join(base_path, 'node-feat.csv.gz'))
edges_csv = pd.read_csv(os.path.join(base_path, 'edge.csv.gz'))
num_edge_list_csv = pd.read_csv(os.path.join(base_path, 'num-edge-list.csv.gz'))
num_node_list_csv = pd.read_csv(os.path.join(base_path, 'num-node-list.csv.gz'))

In [19]:
pt_base_path = './dataset/ogbl_ppa/split/throughput'
train_load = torch.load(os.path.join(pt_base_path, 'train.pt'))
val_load = torch.load(os.path.join(pt_base_path, 'valid.pt'))
test_load = torch.load(os.path.join(pt_base_path, 'test.pt'))

In [20]:
node_feat = torch.tensor(node_feat_csv.values)
edges = torch.tensor(edges_csv.values)

In [21]:
print(node_feat.shape)

torch.Size([576288, 58])


In [135]:
train_idx = 10000
val_idx=20000

In [140]:
#train_edges
train_edges = edges[edges[:,0]<train_idx]
train_edges = train_edges[train_edges[:,1]<train_idx]
#validation edges
val_edges = edges[edges[:,0]<val_idx]
val_edges = val_edges[val_edges[:,1]<val_idx]
val_edges = val_edges[val_edges[:,0]>=train_idx]
val_edges = val_edges[val_edges[:,1]>=train_idx]

In [141]:
print(node_feat.shape)

torch.Size([576288, 58])


In [143]:
adj_matrix = torch.zeros([train_idx, train_idx])
adj_matrix_valid = torch.zeros([val_idx - train_idx, val_idx - train_idx])

In [144]:
train_edges_np = train_edges.numpy()
for edge in train_edges_np:
    node1 = edge[0]
    node2 = edge[1]
    adj_matrix[node1][node2] = 1
    adj_matrix[node2][node1] = 1

In [145]:
val_edges_np = val_edges.numpy()
for edge in val_edges_np:
    node1 = edge[0] - train_idx
    node2 = edge[1] - train_idx
    adj_matrix_valid[node1][node2] = 1
    adj_matrix_valid[node2][node1] = 1

In [152]:
train_features = []
for row in adj_matrix.numpy():
    ones = np.where(row[:]==1)
    for one_vects in ones:
        node_feat_to_agg = node_feat[one_vects].numpy()
        node_feat_sum = np.sum(node_feat_to_agg, axis=0)
        node_feat_sum_list = node_feat_sum.tolist()
        train_features.append(node_feat_sum_list)

In [153]:
val_features = []
for row in adj_matrix_valid.numpy():
    ones = np.where(row[:]==1)
    ones = np.add(ones, train_idx)
    print(ones)
    for one_vects in ones:
        node_feat_to_agg = node_feat[one_vects].numpy()
        node_feat_sum = np.sum(node_feat_to_agg, axis=0)
        node_feat_sum_list = node_feat_sum.tolist()
        val_features.append(node_feat_sum_list)

[]
[[11968 12276 13370 14060 16506 16668 18360 19958]]
[]
[]
[[13463 15613 17685]]
[]
[[10951 11119 11453 11968 12647 12785 14856 15304 15330 16851 17160 17451
  18489 19445]]
[]
[[15643 19619]]
[[13811]]
[]
[]
[[10769]]
[]
[[18444]]
[]
[]
[[19980]]
[[19797]]
[[10397 17465]]
[[14758]]
[[10857 11294]]
[[14282]]
[[14066]]
[[18978]]
[]
[[18605]]
[]
[]
[[11729 12234 12272 12433 14037 14207 16810 19734]]
[[14116 16055 19805]]
[[13582 19566]]
[[12088 12334 12971 16858 18268 18453]]
[[18096]]
[]
[]
[[19037]]
[]
[[13864]]
[]
[[12806 15062 17874]]
[]
[[10951 12819 13124 13230 14083 15084]]
[[15978]]
[[12105 15821]]
[[16313]]
[]
[[10138 11932 15305 15654 19423]]
[[11785]]
[]
[[13309]]
[]
[[10444 13036 14081]]
[[13513]]
[]
[[16375 18483]]
[[10704]]
[[12786]]
[]
[[16294 19279]]
[]
[]
[[16296]]
[[13013 19626]]
[[10273 10288 13243 13381 15297 16744 17096 17421]]
[[10298 10532 11884 19553]]
[[17926]]
[[11053 16764]]
[[13051 19427]]
[[14282]]
[]
[]
[[14035 15388]]
[]
[[15602]]
[[18235 18330]]
[]
[[160

[[17916]]
[]
[]
[[11318 15897]]
[[14733 16476]]
[[10556 16470]]
[[16204]]
[]
[[14764]]
[[15020]]
[[11664 15205]]
[[14914]]
[]
[[13022 14197 18095]]
[[19317]]
[[10230]]
[[10621 16830]]
[]
[]
[]
[]
[]
[[16007 19085]]
[[11905]]
[[16397 19635]]
[[10227 11139 14777 15406]]
[]
[]
[[10906 12032]]
[[18140 18247 18518 18899]]
[[13030 15913 16144 16752 17263 17825 17871]]
[[12999 19399]]
[]
[[15922 18200 18916 19197]]
[[13432]]
[]
[[19997]]
[[11084]]
[[11035 11570 18270]]
[]
[]
[[13660]]
[]
[[13963]]
[]
[]
[]
[]
[]
[[11583 14695 15476]]
[[12269 15274]]
[[11758]]
[]
[[10079 10587 10644 18507]]
[]
[[14824]]
[]
[]
[]
[[14886]]
[]
[[15008]]
[]
[]
[]
[[11336]]
[[19687]]
[[17971]]
[[10021 17296]]
[[10834]]
[]
[]
[]
[]
[[15486 19062]]
[]
[[12233 17538 19788]]
[[12905 17034 17078]]
[[16991]]
[[15382]]
[[11472 18718]]
[]
[]
[]
[]
[[14863]]
[]
[]
[]
[]
[]
[]
[[11229 15897 15994]]
[]
[]
[[13394 17676]]
[]
[]
[]
[[11093 12181 13570 14340 15633 16076]]
[[13060]]
[[13641 14545 19302]]
[[19475]]
[[14325 18210 

[]
[[10411 13003 16868]]
[[14216 14473 15231]]
[]
[[11973 13079 17200]]
[]
[[16244 17554 19387]]
[]
[[10568]]
[]
[[10029 14829]]
[[11175 12005 13440 15271 15544 16231 19326]]
[]
[]
[]
[[12668 13624]]
[[12028]]
[]
[]
[]
[]
[]
[]
[[10746 17679]]
[]
[]
[[17932]]
[[18857]]
[[15173 16484]]
[[11419 11929 12971 14517 15643]]
[[14616 17881 19919]]
[[14700]]
[[10130 14357 18783 19588]]
[[13683 18789]]
[[16355]]
[]
[]
[[18387]]
[]
[[13578 16221]]
[[11219 13891 15756]]
[[10669]]
[]
[[14674 14778 14990 16946 17313 18337]]
[[13367 17210 19830]]
[]
[]
[]
[[17930]]
[[11513]]
[]
[[12303]]
[[15180 17147 17689 17740]]
[[10127 10175 15005 16388]]
[[10158 19091]]
[]
[]
[[15373]]
[[10906 11464 11630 13986 16306 17598 17662 18684 19561]]
[]
[]
[]
[[11699 13438 13506 17706]]
[[12314 17915 19023]]
[[13863 14040 14236 16478 16651 16726]]
[[10151 14813 17540]]
[[10120]]
[]
[]
[[10951 15475 18332]]
[]
[]
[[18236 19182]]
[[12850 18064 18331]]
[]
[[12328 17503 19966]]
[]
[]
[[13622 14143]]
[[13888 15697 18444]]
[]

[]
[]
[[18740 19222]]
[[17159 18844]]
[[15696 18458 18621]]
[]
[[17514]]
[[18123]]
[]
[]
[[14167]]
[[12393 14098 14807]]
[[10127 10655 14812]]
[]
[[11376 18948]]
[[12167]]
[[11934 12456 15697 18362 18969 19230]]
[[11893 12700 14659]]
[[16351]]
[[12001 16215 18094 19421]]
[[11828 18814 19546]]
[[15615]]
[]
[]
[[10951 15457 19390]]
[[14150]]
[]
[]
[]
[[11472 19123]]
[[13111 19027 19185]]
[[15228]]
[[12542]]
[]
[[17564]]
[]
[[10318 11784 16909 17196]]
[]
[[11663 17030]]
[]
[]
[[14779 16920 19848]]
[]
[]
[]
[[12415]]
[]
[[14731]]
[[11990 13384 16030]]
[[16023]]
[[10298 12167 14126 18270 18296]]
[]
[]
[]
[]
[]
[[12953]]
[[12536]]
[[15601]]
[]
[[11119 15305 15654 19220]]
[]
[[10089 10412 12317 14258 14271 16144 18369 19842]]
[]
[[15244]]
[[16292]]
[]
[]
[]
[[10278 11131 11964 17088]]
[[11565]]
[[19454]]
[]
[[17317 18163 19253]]
[[16292]]
[]
[]
[]
[]
[]
[]
[]
[]
[[17895]]
[]
[]
[]
[[10396 11581]]
[]
[[14550]]
[[11057 16104 16711]]
[]
[]
[[12837]]
[]
[[12109]]
[[11219 16005]]
[[11388 12688]]
[

[[10288 14768 19229 19453]]
[]
[[10863 14236 14294 16651]]
[]
[[12905 14213 17205 18549]]
[]
[]
[]
[]
[]
[[16079]]
[]
[]
[]
[]
[[11237]]
[]
[]
[]
[[15941]]
[]
[]
[]
[[10906 18684]]
[[10673 14168 16851 19433 19627]]
[[11762 12382]]
[]
[[11658]]
[]
[]
[[14581]]
[]
[[11118]]
[]
[]
[]
[]
[]
[]
[]
[[10979]]
[]
[[15785]]
[[14768 14794 18441 19678]]
[]
[]
[[13089 17810 17954 19284]]
[[16075]]
[]
[[11092 19411 19732 19736]]
[[15260]]
[]
[[16506 19292]]
[[13567 16560]]
[]
[]
[[13448 15011 15394 15890 18676]]
[]
[]
[]
[]
[[11856]]
[[17797]]
[[10328 10627 17554]]
[]
[]
[]
[[17623]]
[]
[]
[[13461 16518]]
[[13484]]
[]
[]
[]
[]
[[16787]]
[]
[[11677 12956 12973 13255 14054 14624 14705 15691 16261 18069 19166]]
[]
[]
[]
[]
[]
[]
[]
[[15994]]
[]
[[16120]]
[[12674]]
[[12063 19597]]
[[12466 13903 19353]]
[]
[[13642]]
[[13882]]
[]
[[12159 13579 17414 17540]]
[[16579 17323]]
[]
[[19362 19781]]
[]
[[13798]]
[[19202 19805]]
[]
[]
[]
[[10719 11701 12476 12910 14159 15009 16868 17374 17811 19822]]
[]
[[19280]]

[]
[[15238 16954]]
[]
[]
[[13848]]
[[11256 11516 13030 13729 14568 14846 15183 17871 19024]]
[]
[]
[]
[]
[[16319]]
[[16240]]
[]
[[14391 14825 15897 16834 18819]]
[[14437 17671]]
[]
[]
[[13909 19394]]
[]
[]
[[15086 17113]]
[]
[]
[[14101 14616 16004 16066 16831 17771 18723]]
[]
[]
[]
[]
[[11907 12618 13489 15394 16638 16724 16848 18854]]
[]
[[16321]]
[[14583 18896]]
[]
[[19711]]
[[14473]]
[[18818]]
[]
[]
[]
[[14258 17854]]
[[15528 15922 18247 19004]]
[[16119]]
[]
[[10174]]
[]
[[19034]]
[]
[[19449]]
[]
[]
[[13660]]
[]
[]
[[13931]]
[]
[]
[]
[[12063 18070]]
[]
[[10951 16523]]
[[12534 16843 18504]]
[[18512]]
[[15877 17256 17533]]
[[10666 13931 14417 15672 15679 19148]]
[]
[[11232]]
[[18688]]
[[14751]]
[[13156 18101 19871 19880]]
[]
[[16830 19827]]
[]
[]
[]
[]
[[10140 11747 11818 19164]]
[[13003 13686]]
[]
[[14316]]
[]
[]
[[15615]]
[[12462 17770 17889]]
[[10623 15784]]
[]
[[17742]]
[]
[[13432]]
[]
[[13087 17145]]
[[17798]]
[]
[[12434]]
[]
[[15008]]
[[16468]]
[]
[]
[]
[]
[[10990 11477]]
[[1615

[]
[[15005]]
[]
[]
[[15814]]
[]
[[18611]]
[]
[]
[[12927]]
[]
[[14684]]
[]
[]
[[13972]]
[]
[]
[]
[[12592 18327]]
[]
[[15565]]
[[11852 15560 19417]]
[[10852]]
[[14431 15219]]
[]
[[10354 12884 13518 16359]]
[[17698]]
[[13511]]
[[12276 15836 17451 19325]]
[[12685 15691 16278]]
[[14213 14275 18366]]
[[13907 15086 16468 18333]]
[]
[]
[]
[]
[[18983]]
[[13876]]
[]
[[14398]]
[]
[[14995 17540]]
[[10504 15137 16752 18801]]
[]
[[12770]]
[]
[[12034 15226 18177 18423 18876 18962]]
[]
[[10064 14890 15215 15458 16138]]
[]
[[10340 16894]]
[[14523]]
[]
[]
[]
[]
[]
[]
[]
[[19591]]
[]
[[13425]]
[[18600]]
[[10110]]
[[10473 12550]]
[]
[]
[[11620 16084]]
[]
[[10496]]
[]
[]
[]
[]
[]
[]
[[12020 13056 15935]]
[[12388 17784]]
[[10006 11119 11453 12276 14161 15524 15713 15952 17401 17564 19445 19958]]
[[16017 17280]]
[[16627]]
[]
[[19694]]
[]
[]
[[16917]]
[]
[[13994]]
[[19254]]
[[12052]]
[]
[]
[[10019 14699 16974]]
[[19357]]
[]
[[10396 16116 18877]]
[]
[]
[[17786]]
[[14039 17988 18328]]
[]
[]
[[14794]]
[]
[[10965

[[12207 12616 17379]]
[[13405 14030 16077]]
[]
[[11896]]
[]
[]
[]
[[11853 14294]]
[[14359 17625]]
[]
[[13671 15696]]
[]
[]
[]
[[12633 16837 17258]]
[[14540 14798]]
[[12317 14101 16004 16066 16831 17771 18723 19321]]
[[19194]]
[[11165 12298]]
[]
[]
[[10759 11710 15626 18140]]
[[14356 18270]]
[]
[[14700 16727 19164]]
[]
[]
[]
[[14294 18665]]
[]
[[13636 16646]]
[]
[[11448 13989]]
[[13433]]
[[19330]]
[]
[]
[]
[]
[[16386]]
[]
[[11612]]
[[13876]]
[]
[]
[]
[[15279]]
[]
[]
[]
[[16319]]
[]
[[13201 18143]]
[]
[[11733 18639]]
[]
[[10343 12127 12151 14107 16474 16846 17550 17762 18575 18899 19427]]
[]
[]
[[17855]]
[[19067]]
[]
[]
[]
[]
[[14955 15011 15394 15890 17563]]
[]
[]
[[12671]]
[]
[]
[]
[[11200]]
[[10644 10906 11464 12231 12307 12481 12751 12775 14423 14708 14922 16991
  17598 18236 19459]]
[]
[[19272]]
[]
[[16205 19656]]
[]
[]
[[17640]]
[]
[]
[[14575]]
[[12517 14260]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[17062]]
[[10175 18900]]
[]
[]
[]
[]
[[13955 18180 19855]]
[]
[[19356]]
[]
[[10915 14168]]
[

[[13797 16736]]
[[10438 14227]]
[[12414 14608]]
[[10861 19664]]
[]
[]
[[10800 18712]]
[]
[]
[]
[]
[]
[]
[[19830]]
[]
[]
[]
[]
[]
[[10659 12210 13025 13030 14351 14852 15323 15487 15913 17825 18369 19321
  19764]]
[[11002 11761 12910 15953 17180 19344 19737]]
[]
[[12668 16207]]
[[11978 18282]]
[]
[]
[[14695 15234]]
[]
[]
[]
[[10465 18581 19502]]
[[11003 12978 16207]]
[[15205]]
[[14885]]
[]
[]
[[15736]]
[]
[[15572]]
[]
[[13566]]
[[15151]]
[[12404]]
[[16922]]
[[17552]]
[[10224 17071]]
[[13811]]
[]
[]
[[16401]]
[]
[]
[[13918]]
[]
[[18366]]
[]
[[11813 14815]]
[]
[[17679]]
[]
[[13064 13566 19631]]
[]
[]
[[16327]]
[[13566 13628 14015]]
[[11845 14622 18521]]
[[18753]]
[[10592]]
[[13432 14796]]
[]
[[12453]]
[]
[]
[]
[]
[[17809]]
[]
[[10756]]
[]
[]
[[18258]]
[]
[]
[[10762 11160 13145 16794]]
[]
[]
[[11636 19095]]
[[10313]]
[]
[]
[[16020 19187]]
[[19693]]
[[15176]]
[[12042]]
[]
[[16300 18744 19760]]
[]
[]
[[10827 15251 17871]]
[]
[]
[]
[]
[]
[]
[[10951]]
[]
[[11147 11193]]
[]
[[10001 10337 10413 

In [112]:
train_features = np.asarray(train_features)
print(train_features.shape)

(10000, 58)


In [155]:
val_features = np.asarray(val_features)
print(val_features.shape)
print(val_features)

(10000, 58)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [113]:
print(train_features)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [156]:
from src.models import *

In [114]:
'''model = GCNModel(nfeat=58,
                 nhid=128,
                 nclass=58,
                 nhidlayer=1,
                 dropout=0.1,
                 baseblock="mutigcn",
                 inputlayer="gcn",
                 outputlayer="gcn",
                 nbaselayer=0,
                 activation=lambda x: x,
                 withbn=True,
                 withloop=True,
                 aggrmethod="add",
                 mixmode=False)
'''

In [115]:
#creating labels
feature_map = {}
for feature in node_feat.numpy():
    feature_tuple = tuple(feature.tolist())
    if feature_tuple not in feature_map:
        feature_idx = int(np.where(feature[:]==1)[0][0])
        feature_map[feature_tuple] = feature_idx
        print("feature {} mapped to {}".format(feature, feature_idx))


feature [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] mapped to 16
feature [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] mapped to 48
feature [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] mapped to 26
feature [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] mapped to 10
feature [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] mapped to 32
feature [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0] mapped to 50
feature [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] mapped to 39
feature [0 0 0 0 0 0

In [116]:
#checking lengths
feature_counts = []
for feature in node_feat:
    if feature.tolist() not in feature_counts:
        feature_counts.append(feature.tolist())
print(len(feature_counts))
feature_counts.sort()

58


In [117]:
indices = []
for feature in feature_counts:
    indices.append(feature.index(1))
indices.sort()
print(indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [118]:
print(node_feat.shape)

torch.Size([576288, 58])


In [119]:
train_labels = []
features_count = []
for feature in node_feat.numpy()[:train_idx]:
    feature_tuple = tuple(feature)
    label = int(feature_map[feature_tuple])
    train_labels.append(label)
train_labels_array = np.asarray(train_labels)
train_labels_tensor = torch.tensor(train_labels_array)

In [157]:
val_labels = []
features_count = []
for feature in node_feat.numpy()[train_idx:val_idx]:
    feature_tuple = tuple(feature)
    label = int(feature_map[feature_tuple])
    val_labels.append(label)
val_labels_array = np.asarray(val_labels)
val_labels_tensor = torch.tensor(val_labels_array)

In [162]:
labels = train_labels_tensor.type(torch.float).cuda()
val_labels = val_labels_tensor.type(torch.float).cuda()
train_adj = adj_matrix.type(torch.float).cuda()
train_fea = torch.tensor(train_features).type(torch.float).cuda()
val_adj = adj_matrix_valid.type(torch.float).cuda()
val_fea = torch.tensor(val_features).type(torch.float).cuda()

In [121]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter

from src.metric import accuracy, roc_auc_compute_fn
# from deepgcn.utils import load_data, accuracy
# from deepgcn.models import GCN

from src.metric import accuracy
from src.utils import load_citation, load_reddit_data
from src.models import *

In [122]:
model_type = "mutigcn"
if model_type is "mutigcn":
    print("For the multi-layer gcn model, the aggrmethod is fixed to nores and nhiddenlayers = 1.")
    nhiddenlayer = 1
    aggrmethod = "nores"

For the multi-layer gcn model, the aggrmethod is fixed to nores and nhiddenlayers = 1.


In [123]:
cuda = torch.cuda.is_available()
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [124]:
nfeat = 58
nclass = 58
#False False concat False

In [225]:

model = GCNModel(nfeat=58,
                 nhid=128,
                 nclass=58,
                 nhidlayer=1,
                 dropout=0.5,
                 baseblock='inceptiongcn',
                 inputlayer='gcn',
                 outputlayer='gcn',
                 nbaselayer=6,
                 activation=F.relu,
                 withbn=False,
                 withloop=False,
                 aggrmethod='concat',
                 mixmode=False)

In [226]:
optimizer = torch.optim.Adam(model.parameters(),
                       lr=0.001, weight_decay=5e-5)

In [227]:
model.cuda()

GCNModel(
  (ingc): GraphConvolutionBS (58 -> 128)
  (outgc): GraphConvolutionBS (896 -> 58)
  (midlayer): ModuleList(
    (0): InecptionGCNBlock concat (128 - [128:6] > 896)
  )
)

In [228]:
model.load_state_dict(torch.load('/future/u/alexder/other/DropEdgeOriginal/DropEdge/tmpmodel/model.pth'))

<All keys matched successfully>

In [229]:
print(train_fea.shape)
device = torch.device("cuda:0")

torch.Size([10000, 58])


In [230]:
def train(epoch, train_adj, train_fea, val_adj=None, val_fea=None):
    '''if val_adj is None:
        val_adj = train_adj
        val_fea = train_fea
    '''
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(train_fea, train_adj)
    # special for reddit
    
    loss_train = F.nll_loss(output, labels.type(torch.long))
    acc_train = accuracy(output, labels)

    loss_train.backward()
    optimizer.step()
    train_t = time.time() - t
    val_t = time.time()
    # We can not apply the fastmode for the reddit dataset.
    # if sampler.learning_type == "inductive" or not args.fastmode:
    '''
    if args.early_stopping > 0 and sampler.dataset != "reddit":
        loss_val = F.nll_loss(output[idx_val], labels[idx_val]).item()
        early_stopping(loss_val, model)

    if not args.fastmode:
        #    # Evaluate validation set performance separately,
        #    # deactivates dropout during validation run.
        model.eval()
        output = model(val_fea, val_adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val]).item()
        acc_val = accuracy(output[idx_val], labels[idx_val]).item()
        if sampler.dataset == "reddit":
            early_stopping(loss_val, model)
    else:
    '''
    model.eval()
    output_valid = model(val_fea, val_adj)
    loss_val = F.nll_loss(output_valid, val_labels.type(torch.long)).item()
    acc_val = accuracy(output_valid, val_labels).item()
    
    #loss_val = 0
    #acc_val = 0

    '''
    if args.lradjust:
        scheduler.step()
    '''

    val_t = time.time() - val_t
    return (loss_train.item(), acc_train.item(), loss_val, acc_val, train_t, val_t)

In [231]:
import time
epochs=1000
for epoch in range(epochs):
    sampling_t = time.time()
    # no sampling
    # randomedge sampling if args.sampling_percent >= 1.0, it behaves the same as stub_sampler.

    sampling_t = time.time() - sampling_t

    # The validation set is controlled by idx_val
    # if sampler.learning_type == "transductive":
    #train_fea_rand = torch.rand([10000, 58]).cuda()
    outputs = train(epoch, train_adj, train_fea, val_adj, val_fea)
    
    if epoch % 10 == 0:
        print('Epoch: {:04d}'.format(epoch + 1),
              'loss_train: {:.4f}'.format(outputs[0]),
              'acc_train: {:.4f}'.format(outputs[1]),
              'loss_val: {:.4f}'.format(outputs[2]),
              'acc_val: {:.4f}'.format(outputs[3]),
              's_time: {:.4f}s'.format(sampling_t),
              't_time: {:.4f}s'.format(outputs[4]),
              'v_time: {:.4f}s'.format(outputs[5]))
    '''
    if args.no_tensorboard is False:
        tb_writer.add_scalars('Loss', {'train': outputs[0], 'val': outputs[2]}, epoch)
        tb_writer.add_scalars('Accuracy', {'train': outputs[1], 'val': outputs[3]}, epoch)
        tb_writer.add_scalar('lr', outputs[4], epoch)
        tb_writer.add_scalars('Time', {'train': outputs[5], 'val': outputs[6]}, epoch)


    loss_train[epoch], acc_train[epoch], loss_val[epoch], acc_val[epoch] = outputs[0], outputs[1], outputs[2], outputs[
        3]

    if args.early_stopping > 0 and early_stopping.early_stop:
        print("Early stopping.")
        model.load_state_dict(early_stopping.load_checkpoint())
        break
    '''

Epoch: 0001 loss_train: 1939.1808 acc_train: 0.0234 loss_val: 334.3433 acc_val: 0.0310 s_time: 0.0000s t_time: 0.0290s v_time: 0.1288s
Epoch: 0011 loss_train: 338.0061 acc_train: 0.0334 loss_val: 136.2746 acc_val: 0.0396 s_time: 0.0000s t_time: 0.0145s v_time: 0.1273s
Epoch: 0021 loss_train: 121.7497 acc_train: 0.0342 loss_val: 54.6329 acc_val: 0.0379 s_time: 0.0000s t_time: 0.0140s v_time: 0.1278s
Epoch: 0031 loss_train: 91.5112 acc_train: 0.0407 loss_val: 33.8442 acc_val: 0.0487 s_time: 0.0000s t_time: 0.0140s v_time: 0.1277s
Epoch: 0041 loss_train: 47.0952 acc_train: 0.0436 loss_val: 27.3464 acc_val: 0.0494 s_time: 0.0000s t_time: 0.0140s v_time: 0.1277s
Epoch: 0051 loss_train: 52.6158 acc_train: 0.0506 loss_val: 25.7167 acc_val: 0.0547 s_time: 0.0000s t_time: 0.0141s v_time: 0.1277s
Epoch: 0061 loss_train: 29.9903 acc_train: 0.0516 loss_val: 17.7233 acc_val: 0.0621 s_time: 0.0000s t_time: 0.0141s v_time: 0.1277s
Epoch: 0071 loss_train: 25.0810 acc_train: 0.0630 loss_val: 18.0733 ac

Epoch: 0631 loss_train: 3.1609 acc_train: 0.2034 loss_val: 3.4987 acc_val: 0.2214 s_time: 0.0000s t_time: 0.0149s v_time: 0.1267s
Epoch: 0641 loss_train: 3.1761 acc_train: 0.2067 loss_val: 3.5038 acc_val: 0.2240 s_time: 0.0000s t_time: 0.0155s v_time: 0.1261s
Epoch: 0651 loss_train: 3.1403 acc_train: 0.2071 loss_val: 3.5050 acc_val: 0.2321 s_time: 0.0000s t_time: 0.0146s v_time: 0.1271s
Epoch: 0661 loss_train: 3.1796 acc_train: 0.2111 loss_val: 3.5023 acc_val: 0.2364 s_time: 0.0000s t_time: 0.0140s v_time: 0.1277s
Epoch: 0671 loss_train: 3.1571 acc_train: 0.2096 loss_val: 3.4797 acc_val: 0.2374 s_time: 0.0000s t_time: 0.0140s v_time: 0.1277s
Epoch: 0681 loss_train: 3.1355 acc_train: 0.2119 loss_val: 3.4288 acc_val: 0.2397 s_time: 0.0000s t_time: 0.0141s v_time: 0.1276s
Epoch: 0691 loss_train: 3.1269 acc_train: 0.2153 loss_val: 3.3807 acc_val: 0.2381 s_time: 0.0000s t_time: 0.0144s v_time: 0.1273s
Epoch: 0701 loss_train: 3.1292 acc_train: 0.2134 loss_val: 3.3730 acc_val: 0.2408 s_time: 

In [241]:
#DROPEDGE_IMPLEMENTATION:
train_adj_ones = (train_adj==1).nonzero()
train_adj_ones = train_adj_ones[train_adj_ones[:,0] < train_adj_ones[:,1]]
nb_nonzero_entries =  train_adj_ones.shape[0]
DROPEDGE_P = 0.2

In [242]:
import random
drop_probs = [random.random() for x in range(nb_nonzero_entries)]
to_drop = [i for i in range(len(drop_probs)) if drop_probs[i]<DROPEDGE_P]

In [243]:
train_adj_dropedge = train_adj
for i in to_drop:
    coords = train_adj_ones[i]
    node1 = coords[0]
    node2 = coords[1]
    train_adj_dropedge[node1][node2] = 0
    train_adj_dropedge[node2][node1] = 0
print(train_adj_dropedge.shape)

torch.Size([10000, 10000])


In [244]:
#train with dropedge

In [245]:

model = GCNModel(nfeat=58,
                 nhid=128,
                 nclass=58,
                 nhidlayer=1,
                 dropout=0.5,
                 baseblock='inceptiongcn',
                 inputlayer='gcn',
                 outputlayer='gcn',
                 nbaselayer=6,
                 activation=F.relu,
                 withbn=False,
                 withloop=False,
                 aggrmethod='concat',
                 mixmode=False)
model.cuda()
model.load_state_dict(torch.load('/future/u/alexder/other/DropEdgeOriginal/DropEdge/tmpmodel/model.pth'))

<All keys matched successfully>

In [246]:
optimizer = torch.optim.Adam(model.parameters(),
                       lr=0.001, weight_decay=5e-5)

In [247]:
import time
epochs=1000
for epoch in range(epochs):
    sampling_t = time.time()
    # no sampling
    # randomedge sampling if args.sampling_percent >= 1.0, it behaves the same as stub_sampler.

    sampling_t = time.time() - sampling_t

    # The validation set is controlled by idx_val
    # if sampler.learning_type == "transductive":
    #train_fea_rand = torch.rand([10000, 58]).cuda()
    outputs = train(epoch, train_adj_dropedge, train_fea, val_adj, val_fea)
    
    if epoch % 10 == 0:
        print('Epoch: {:04d}'.format(epoch + 1),
              'loss_train: {:.4f}'.format(outputs[0]),
              'acc_train: {:.4f}'.format(outputs[1]),
              'loss_val: {:.4f}'.format(outputs[2]),
              'acc_val: {:.4f}'.format(outputs[3]),
              's_time: {:.4f}s'.format(sampling_t),
              't_time: {:.4f}s'.format(outputs[4]),
              'v_time: {:.4f}s'.format(outputs[5]))
    '''
    if args.no_tensorboard is False:
        tb_writer.add_scalars('Loss', {'train': outputs[0], 'val': outputs[2]}, epoch)
        tb_writer.add_scalars('Accuracy', {'train': outputs[1], 'val': outputs[3]}, epoch)
        tb_writer.add_scalar('lr', outputs[4], epoch)
        tb_writer.add_scalars('Time', {'train': outputs[5], 'val': outputs[6]}, epoch)


    loss_train[epoch], acc_train[epoch], loss_val[epoch], acc_val[epoch] = outputs[0], outputs[1], outputs[2], outputs[
        3]

    if args.early_stopping > 0 and early_stopping.early_stop:
        print("Early stopping.")
        model.load_state_dict(early_stopping.load_checkpoint())
        break
    '''

Epoch: 0001 loss_train: 320.0791 acc_train: 0.0236 loss_val: 307.5806 acc_val: 0.0371 s_time: 0.0000s t_time: 0.0289s v_time: 0.1283s
Epoch: 0011 loss_train: 62.3396 acc_train: 0.0338 loss_val: 110.7803 acc_val: 0.0383 s_time: 0.0000s t_time: 0.0143s v_time: 0.1275s
Epoch: 0021 loss_train: 30.2937 acc_train: 0.0366 loss_val: 61.0510 acc_val: 0.0456 s_time: 0.0000s t_time: 0.0145s v_time: 0.1274s
Epoch: 0031 loss_train: 15.9797 acc_train: 0.0477 loss_val: 29.0379 acc_val: 0.0619 s_time: 0.0000s t_time: 0.0143s v_time: 0.1275s
Epoch: 0041 loss_train: 10.5748 acc_train: 0.0496 loss_val: 21.4230 acc_val: 0.0619 s_time: 0.0000s t_time: 0.0144s v_time: 0.1274s
Epoch: 0051 loss_train: 7.3646 acc_train: 0.0632 loss_val: 20.2138 acc_val: 0.0790 s_time: 0.0000s t_time: 0.0143s v_time: 0.1275s
Epoch: 0061 loss_train: 7.1699 acc_train: 0.0673 loss_val: 20.0923 acc_val: 0.0842 s_time: 0.0000s t_time: 0.0144s v_time: 0.1274s
Epoch: 0071 loss_train: 7.1179 acc_train: 0.0725 loss_val: 15.2647 acc_val:

Epoch: 0631 loss_train: 3.0836 acc_train: 0.2154 loss_val: 4.6189 acc_val: 0.2681 s_time: 0.0000s t_time: 0.0148s v_time: 0.1268s
Epoch: 0641 loss_train: 3.0833 acc_train: 0.2134 loss_val: 4.5391 acc_val: 0.2679 s_time: 0.0000s t_time: 0.0141s v_time: 0.1276s
Epoch: 0651 loss_train: 3.0710 acc_train: 0.2156 loss_val: 4.3387 acc_val: 0.2697 s_time: 0.0000s t_time: 0.0139s v_time: 0.1278s
Epoch: 0661 loss_train: 3.0618 acc_train: 0.2172 loss_val: 4.1722 acc_val: 0.2707 s_time: 0.0000s t_time: 0.0140s v_time: 0.1277s
Epoch: 0671 loss_train: 3.0432 acc_train: 0.2227 loss_val: 4.0887 acc_val: 0.2705 s_time: 0.0000s t_time: 0.0233s v_time: 0.1184s
Epoch: 0681 loss_train: 3.0514 acc_train: 0.2196 loss_val: 4.1459 acc_val: 0.2703 s_time: 0.0000s t_time: 0.0152s v_time: 0.1265s
Epoch: 0691 loss_train: 3.0422 acc_train: 0.2222 loss_val: 4.2342 acc_val: 0.2712 s_time: 0.0000s t_time: 0.0154s v_time: 0.1262s
Epoch: 0701 loss_train: 3.0454 acc_train: 0.2194 loss_val: 4.3047 acc_val: 0.2721 s_time: 